In [ ]:
#default_exp utils.stacked_dataloader

In [ ]:
#export
"Dataloader wrapper that can combine and handle multiple dataloaders for multitask training"

# DataLoading
class StackedDLs():
    def __init__(self, dls, num_it=100):
        self.dls = dls
        self.train = StackedDL([dls.train for dls in self.dls], num_it)
        self.valid = StackedDL([dls.valid for dls in self.dls], num_it)
        self.path = dls[0].path
        self.device = dls[0].device
        self.vocab = dls[0].vocab
        self.empty_val = False
        self.n_inp = 1
        
    def __getitem__(self, i):
        return [self.train, self.valid][i]
    def __getattr__(self, attr):
        def redirected(*args, **kwargs):
            vals = []
            for dl in self.dls:
                if hasattr(dl, attr):
                    val = getattr(dl, attr)(*args, **kwargs)
                    vals.append(val)
            if vals: return vals[0]
        if any([hasattr(dl, attr) for dl in self.dls]): return redirected
        raise AttributeError
    
class StackedDL():
    def __init__(self, dls, num_it=100):
        self.dls = dls
        self.num_it = num_it
        self.dl_idx = -1
        self.n_inp = 1
        
    def __len__(self)->int: return sum([len(dl) for dl in self.dls])
    def __getattr__(self, attr):
        def redirected(*args, **kwargs):
            vals = []
            for dl in self.dls:
                if hasattr(dl, attr):
                    val = getattr(dl, attr)(*args, **kwargs)
                    vals.append(val)
            if vals: return vals[0]
        if any([hasattr(dl, attr) for dl in self.dls]): return redirected
        raise AttributeError
        
    def __iter__(self):
        "Process and returns items from `DataLoader`."
        iters = [iter(dl) for dl in self.dls]
        self.dl_idx = -1
        while len(iters):
            self.dl_idx = (self.dl_idx+1) % len(iters)
            for b in range(self.num_it):
                try:
                    yield next(iters[self.dl_idx])
                except StopIteration as e:
                    iters.remove(iters[self.dl_idx])
                    break

In [1]:
#hide
from nbdev.export import notebook2script
notebook2script(recursive=True)

Converted config.ipynb.
Converted Train-before_cleanup.ipynb.
Converted Train.ipynb.
Converted dataloader.ipynb.
Converted learner.ipynb.
Converted model.ipynb.
Converted transform.ipynb.
Converted Train.ipynb.
Converted dataloader.ipynb.
Converted learner.ipynb.
Converted model.ipynb.
Converted Train-Scratch.ipynb.
Converted dataloader-reference.ipynb.
Converted dataloader-v1.ipynb.
Converted transform.ipynb.
Converted numpy_encode.ipynb.
Converted attention_mask.ipynb.
Converted env_setup.ipynb.
Converted fastai_transformer.ipynb.
Converted file_processing.ipynb.
Converted lamb.ipynb.
Converted midifile.ipynb.
Converted stacked_dataloader.ipynb.
Converted top_k_top_p.ipynb.
Converted vocab.ipynb.
